# Example for running HiggsSignals to constrain the Higgs $\text{BR}(h_{125}\to\text{NP})$ as a function of $c_\text{eff}$

In [ ]:
import Higgs.predictions as HP
import Higgs.signals as HS
import numpy as np
import pandas as pd

pred = HP.Predictions() # create the model predictions
signals = HS.Signals('/Path/To/HSDataSet') # load HS dataset

### HiggsPredictions input

In [ ]:
# add a SM-like particle
h = pred.addParticle(HP.NeutralScalar("h", "even"))
h.setMass(125.09)

In [ ]:
# function to set h couplings to SM particles, which are enhanced w.r.t. to the SM by the factor ceff
def setEffC(ceff):
    HP.effectiveCouplingInput(h, HP.scaledSMlikeEffCouplings(ceff), reference="SMHiggsEW")

# function to set an additional BR to new phyics
def setBRNP(BRNP):
    tw_before = h.totalWidth()
    dw_NP = BRNP * tw_before / (1 - BRNP)
    h.setDecayWidth('dummy', 'dummy', dw_NP)
    tw_new = h.totalWidth()
    return tw_new

### Run HiggsSignals

setup scanning ranges

In [ ]:
df = pd.DataFrame()

N = 150

x = np.linspace(0, 5, N)
y = np.linspace(0, 0.99, N)
X, Y = np.meshgrid(x, y)

df['ceff'] = X.flatten()
df['BRNP'] = Y.flatten()

derive HS results

In [ ]:
@np.vectorize
def calcChisqWidth(ceff, BRNP):
    setEffC(ceff)
    tw = setBRNP(BRNP)
    return signals(pred), tw

In [ ]:
df['chisq'], df['tw'] = calcChisqWidth(df['ceff'], df['BRNP'])

calculate $\Delta\chi^2$ and cut-off all values above $8.01$ for plotting

In [ ]:
df['dchisq']   = df['chisq'] - df['chisq'].min()

bound = 8.01

@np.vectorize
def cutChisq(e):
    if e < bound:
        return e
    else:
        return bound

df['dchisq'] = cutChisq(df['dchisq'])

### Plotting

In [ ]:
from scipy import interpolate
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'

In [ ]:
fig, ax = plt.subplots(
        figsize=(3.0, 3.0),
        constrained_layout=True)

Z   = interpolate.griddata((df['ceff'], df['BRNP']), df['dchisq'], (X, Y), method='linear')
Ztw = interpolate.griddata((df['ceff'], df['BRNP']), df['tw'], (X, Y), method='linear')

pcm = ax.pcolormesh(
        X, Y, Z,
        cmap='GnBu_r',
        rasterized=True
)

clb = fig.colorbar(
    pcm,
    ax=ax,
    label=r"$\Delta\chi^2$",
    ticks=np.arange(0, bound, 1),
    pad=0,
    fraction=0.1,
    location='top',
    aspect=40
)
clb.ax.minorticks_on()

ax.contour(
    X, Y, Z,
    levels=[2.3, 5.99, 9.21],
    colors="k",
    linestyles=["-", "--", ":"],
)

ax.contour(
    X, Y, Ztw,
    levels=[4.1e-3, 4.1e-2, 4.1e-1],
    colors="red",
    linestyles=["-", "--", ":"],
)

bfp = np.unravel_index( np.argmin(Z), Z.shape)
ax.plot( x[bfp[1]], y[bfp[0]], marker="*", ls="none", c="orange")

ax.set_xlim([0, 5])
ax.set_ylim([0, 1])

percent = "\%" if plt.rcParams["text.usetex"] else "%"
hsLegend = ax.legend(
    handles=[
        Line2D([0], [0], color="k", ls="-", label=f"HS 68{percent} CL"),
        Line2D([0], [0], color="k", ls="--", label=f"HS 95{percent} CL"),
        Line2D([0], [0], color="orange", ls="none", marker="*", label="HS BFP"),
        Line2D([0], [0], color="red", ls="-", label=r"$\Gamma_h^{\mathrm{SM}}$"),
        Line2D([0], [0], color="red", ls="--", label=r"$10\ \Gamma_h^{\mathrm{SM}}$"),
        Line2D([0], [0], color="red", ls=":", label=r"$100\ \Gamma_h^{\mathrm{SM}}$"),
    ],
    loc="lower right",
    frameon=True,
    ncol=1
)
ax.add_artist(hsLegend)

ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(.2))
ax.yaxis.set_minor_locator(MultipleLocator(.02))

ax.set_xlabel(r'$c_{\rm{eff}}$')
ax.set_ylabel(r'$\rm{BR}$$(h_{125}\to\rm{NP})$')

ax.text(
    0.03,
    0.97,
    r"\texttt{HiggsSignals}"
    if plt.rcParams["text.usetex"]
    else r"$\mathrm{HiggsSignals}$",
    horizontalalignment="left",
    verticalalignment="top",
    transform=ax.transAxes,
)

plt.savefig('brnp.pdf')
plt.show()